### Init Context

In [1]:
from thetaray.api.context import init_context
from datetime import datetime
import yaml
import pyspark.sql.functions as f

import logging
logging.basicConfig(level=logging.DEBUG, format='%(message)s')

with open('/thetaray/git/solutions/domains/demo_digital_wallets/config/spark_config.yaml') as spark_config_file:
    spark_config = yaml.load(spark_config_file, yaml.FullLoader)['spark_config_a']

context = init_context(
    execution_date=datetime(1970, 1, 1),
    #spark_conf=spark_config,
    spark_conf=spark_config, # quitar
    # spark_master='local[*]', # quitar
)


/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
2025-08-19 13:45:10,563:INFO:thetaray.common.logging:start loading solution.....[ load_risks=True , solution_path=/thetaray/git/solutions/domains , settings_path=/thetaray/git/solutions/settings ]
2025-08-19 13:45:10,572:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_human_trafficking_insights, but data encryption is disabled in deployment
2025-08-19 13:45:10,587:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_remittance_customer_insights, but data encryption is disabled in deployment
2025-08-19 13:45:10,597:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_ret_smb_customer_insights, but da

### Params

In [2]:
from thetaray.api.dataset import dataset_functions
from thetaray.api.evaluation import evaluate, write_evaluated_activities, publish_evaluated_activities, load_evaluated_activities
from thetaray.common.data_environment import DataEnvironment

from domains.demo_digital_wallets.datasets.customer_monthly import customer_monthly_dataset as dataset
from domains.demo_digital_wallets.evaluation_flows.ef import evaluation_flow as ef

### Detection

In [3]:
data = dataset_functions.read(context, 
                              dataset().identifier,
                              data_environment=DataEnvironment.PUBLIC).drop('tr_timestamp')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/19 13:45:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/19 13:45:15 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
Hive Session ID = 3ad098ab-957a-4610-87cb-5b403c087569
25/08/19 13:45:19 INFO SessionState: Hive Session ID = 3ad098ab-957a-4610-87cb-5b403c087569


In [4]:
print("Number of transactions in 3_detection:", data.count())

[Stage 4:>                                                          (0 + 5) / 5]

Cantidad de transacciones en 3_detection: 18000


In [5]:
evaluated_data = evaluate(context, 
                          ef().identifier, 
                          data,
                          data_environment=DataEnvironment.PUBLIC)

2025-08-19 13:45:28,737:INFO:thetaray.common.logging:Applying backwards compatibility patches
2025-08-19 13:45:29.047086: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-19 13:45:29.048944: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-19 13:45:29.052056: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-19 13:45:29.060639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755611129.074722   33834 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to re

2025-08-19 13:45:31,694:INFO:thetaray.common.logging:Rolling back backwards compatibility patches
2025-08-19 13:45:31,695:INFO:thetaray.common.logging:Rolled back backwards compatibility patches
2025-08-19 13:45:31,695:INFO:thetaray.common.logging:Applying backwards compatibility patches
2025-08-19 13:45:31,699:INFO:thetaray.common.logging:Applied backwards compatibility patches


2025-08-19 13:45:32,017:INFO:thetaray.common.logging:Rolling back backwards compatibility patches
2025-08-19 13:45:32,018:INFO:thetaray.common.logging:Rolled back backwards compatibility patches


In [7]:
evaluated_data.show(5,0)

25/08/19 13:46:48 ERROR TaskSetManager: Task 0 in stage 8.0 failed 4 times; aborting job


PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/thetaray/platform/python/thetaray/api/evaluation/_scoring.py", line 301, in scoring_function
    features_ratings: ndarray = detection_model.feature_ratings(features_df)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/thetaray/platform/python/thetaray/api/anomaly_detection/anomaly.py", line 97, in feature_ratings
    return self.detector.feature_rating(self.scores)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "Python/ModularSystem/AnalysisModules/AnomalyDetection/AnomalyDetector_SuperClass.py", line 234, in Python.ModularSystem.AnalysisModules.AnomalyDetection.AnomalyDetector_SuperClass.__super_anomaly_detector_old_api__.feature_rating
  File "Python/ModularSystem/AnalysisModules/AnomalyDetection/AnomalyDetector_SuperClass.py", line 88, in Python.ModularSystem.AnalysisModules.AnomalyDetecti

In [6]:
# Total de clientes por mes
df_clients_month = evaluated_data.groupBy('year_month', 'year_month_str') \
    .count() \
    .orderBy('year_month') \
    .toPandas()

# Total de alertas por mes
df_alerts_months = evaluated_data.filter(f.col('algo_score') >= 0.5) \
    .groupBy('year_month') \
    .count() \
    .withColumnRenamed('count', 'count_alerts') \
    .orderBy('year_month') \
    .toPandas()

# Merge y cálculo de porcentaje de alertados
df_clients_alerts = df_clients_month.merge(df_alerts_months, how='left', on='year_month')
df_clients_alerts['count_alerts'] = df_clients_alerts['count_alerts'].fillna(0)
df_clients_alerts['pct_alerted'] = 100 * df_clients_alerts['count_alerts'] / df_clients_alerts['count']

# Mostrar
df_clients_alerts

25/08/19 13:46:04 ERROR TaskSetManager: Task 4 in stage 6.0 failed 4 times; aborting job
/opt/spark/python/pyspark/sql/pandas/conversion.py:198: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.pyspark.fallback.enabled' does not have an effect on failures in the middle of computation.
  
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/thetaray/platform/python/thetaray/api/evaluation/_scoring.py", line 301, in scoring_function
    features_ratings: ndarray = detection_model.feature_ratings(features_df)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/thetaray/platform/python/thetaray/api/anomaly_detection/anomaly.py", line 97, in feature_ratings
    return self.detector.feature_rating(self.scores)
           ^^^^^

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/thetaray/platform/python/thetaray/api/evaluation/_scoring.py", line 301, in scoring_function
    features_ratings: ndarray = detection_model.feature_ratings(features_df)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/thetaray/platform/python/thetaray/api/anomaly_detection/anomaly.py", line 97, in feature_ratings
    return self.detector.feature_rating(self.scores)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "Python/ModularSystem/AnalysisModules/AnomalyDetection/AnomalyDetector_SuperClass.py", line 234, in Python.ModularSystem.AnalysisModules.AnomalyDetection.AnomalyDetector_SuperClass.__super_anomaly_detector_old_api__.feature_rating
  File "Python/ModularSystem/AnalysisModules/AnomalyDetection/AnomalyDetector_SuperClass.py", line 88, in Python.ModularSystem.AnalysisModules.AnomalyDetecti

[Stage 6:>                                                          (0 + 2) / 5]

In [ ]:
from thetaray.api.evaluation import unblock_evaluation_for_execution_date


unblock_evaluation_for_execution_date(
    context,
    evaluation_flow_identifier=ef().identifier,
    data_environment=DataEnvironment.PUBLIC)

In [ ]:
write_evaluated_activities(context, 
                           evaluated_data, 
                           ef().identifier,
                           data_environment=DataEnvironment.PUBLIC)

In [ ]:
publish_evaluated_activities(context, 
                             ef().identifier,
                             data_environment=DataEnvironment.PUBLIC)

In [ ]:
context.close()